In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Nov 19 08:29:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install sacremoses

     |████████████████████████████████| 1.3MB 5.6MB/s 
     |████████████████████████████████| 890kB 12.8MB/s 
     |████████████████████████████████| 1.1MB 28.2MB/s 
     |████████████████████████████████| 2.9MB 38.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=620dc9823a159ff66dc43c4bf780b208959d8e37e90689df9884b533b37f9781
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# PTT gossip classification

這章節我們使用中文預訓練模型 `bert-base-chinese` 來進行 `finetune` 。

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import train_test_split
from transformers import TFBertForSequenceClassification, BertTokenizer, glue_convert_examples_to_features

os.chdir('/content/drive/Shareddrives/類技術班教材/標準版/NLP進階/Seq2seq 系列模型/4.Transformer_based_model/Finetune_on_ptt')

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-chinese')

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

### Data overview

我們使用從 ptt 八卦版進行爬蟲整理，$0$ 表示該留言的推數小於噓數，$1$ 表示該留言的推數大於噓數，所以這個任務是屬於 `Text classification` 任務 (二元分類)。

In [ ]:
ptt = pd.read_csv('Data/ptt_gossip.csv')

bert_max_length = 512
ptt['sentence'] = [t[:bert_max_length] for t in ptt.sentence]

In [ ]:
ptt.head()

,idx,sentence,label
0,0,反核人士最愛靠妖 核電廠蓋你家好不好，我當然說好 核廢料放我家好不好，我也ok 放在地下室就...,1
1,1,如標題， 今天去逛才看到的，如下圖所示: 位置在西屯區漢口路二段118號。 少了一個可以看...,1
2,2,新聞來源 2025非核家園，燃煤電廠30%、再生能源(綠能)20%、 天然氣發電50%的能...,1
3,3,牽了一台新的摩托車 車行老闆跟我說記得汽油要加95 還附帶 我開車行幾十年了 聽我的準沒錯 ...,1
4,4,各位30cm大大、F cup的水水，打給後 胎嘎後 本魯邊緣人，平日臉書沒朋友近日更4鬼怪肆...,1


In [ ]:
"""
訓練集80%，測試集20%
"""
train_size = 0.8

mask = np.random.rand(len(ptt)) < train_size
train_dataset = ptt[mask]
valid_dataset = ptt[~mask]

In [ ]:
train_size = len(train_dataset)
valid_size = len(valid_dataset)

In [ ]:
print('Train size: ', train_size)
print('Valid size: ', valid_size)

Train size:  5656
Valid size:  1419


### Convert to tensor

大部分 `Transformer` 預訓練模型都支持 `tf.tensor` 輸入格式，需要將資料集轉為`tf.tensor`格式。

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(dict(train_dataset))
valid_dataset = tf.data.Dataset.from_tensor_slices(dict(valid_dataset))

### Traing data format

使用 `glue_convert_examples_to_features` 將資料集轉為模型可讀取格式，因為是二元分類，所以我們使用的任務為 `cola`，`cola` 是 `bert` 在 `finetune` 時的任務之一，一樣是二元分類任務，我們可以套用他的輸入格式來進行轉換，而在中文部分目前的預訓練模型都是用 `chararcter-level` 進行斷詞，所以我們將 `max_length` 提高至 $256$，下表為在 `Titan X 12G` 上 `finetune` 的參數限制，表示模型以及多少句子長度對應其最大的`batch_size`，需要注意其硬體限制。

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1eudsSECQlf8SnfqAkJ0w65o8uwd_FnjW' width="300"/>
<figcaption>Self-attention</figcaption></center>
</figure>

接下來我們需要將資料集轉換成模型可讀取的格式，輸入格式有三個：

* `input_ids`: 這表示句子斷完詞之後轉成 `token embeddings`，每一個詞有一個 `id`，如下圖，其中 `101` 表示 `[CLS]`，`102` 表示 `[SEP]`，因為 `MPRC` 是 `Sentence-Pair classification` 任務，所以下面的範例中會看到兩個 `102`。

* `attention mask`: 因為 `Transformer` 會限制輸入句子的長度，最大限制為 `512`，而我們選擇 `128`，但不是所有的句子長度都是 `128`，所以需要在後面進行 `padding` (就是補0)，最主要的目的是不去計算 `padding` 位置的 `loss` 。

* `token_type_ids`: 用來表示 `Segment embedding`，如上圖，表示詞屬於哪一個句子，因為 `MRPC` 有兩個句子，所以 `ids` 有2種，`0` 和 `1`。

In [ ]:
max_length = 512
task = 'cola'

train_dataset = glue_convert_examples_to_features(train_dataset,
                                                  tokenizer,
                                                  max_length,
                                                  task)
valid_dataset = glue_convert_examples_to_features(valid_dataset,
                                                  tokenizer,
                                                  max_length,
                                                  task)

/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:284: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


In [ ]:
train_temp = next(iter(train_dataset))

In [ ]:
train_temp

({'attention_mask': <tf.Tensor: shape=(512,), dtype=int32, numpy=
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0

### Parameter settings

在 `tf.data.Dataset` 中，通常會在訓練資料集後面接上三個標準的操作：

* `.shuffle()`: 打亂資料集的方式，會先從資料集中隨機抽取`buffer_size`筆資料進去 `buffer`，然後再 `buffer` 從中抽取`batch_size`筆資料進行訓練，丟進 `buffer` 的步驟主要是在處理無法一次將所有資料集丟進記憶體進行訓練的情形。

* `.batch()`: 每次迭代使用的資料數量。
* `.repeat()`: `epochs` 數量。

In [ ]:
buffer_size = 100
train_bz = 12
epochs = 3
valid_bz = 12

train_gen = train_dataset.shuffle(buffer_size).batch(train_bz).repeat(epochs)
valid_gen = valid_dataset.batch(valid_bz)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5,
                                     epsilon=1e-8,
                                     clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                     reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

## Training

* `.fit()`: 支援 `generator` 的輸入方式，也可以用 `fit_generator` 。
* `steps_per_epoch`: 每個 `epoch` 訓練幾次，通常是 $\frac{train\_size}{batch\_size}$ ，遍歷整個訓練集。
* `validation_steps`: 與 `steps_per_epoch` 同義。

In [ ]:
history = model.fit(train_gen,
                    epochs=epochs,
                    steps_per_epoch=train_size//train_bz, 
                    validation_data=valid_gen,
                    validation_steps=valid_size//valid_bz)

Epoch 1/3
471/471 [==============================] - 272s 577ms/step - loss: 0.1761 - accuracy: 0.9517 - val_loss: 0.1497 - val_accuracy: 0.9484
Epoch 2/3
471/471 [==============================] - 268s 569ms/step - loss: 0.1249 - accuracy: 0.9642 - val_loss: 0.1361 - val_accuracy: 0.9668
Epoch 3/3
471/471 [==============================] - 269s 572ms/step - loss: 0.0850 - accuracy: 0.9750 - val_loss: 0.1467 - val_accuracy: 0.9654


## Save model

In [ ]:
save_path = 'save_chinese'
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [ ]:
model.save_pretrained('./save_chinese/')

## Evaluation

畫出`precision`, `recall`, `f1-score`以及`confusion matrix`評估模型表現。

In [ ]:
valid_pred = model.predict(valid_gen)
valid_pred_ids = np.argmax(valid_pred[0], axis=-1)

In [ ]:
import numpy as np
"""
從 tf.data.Dataset 中拿取 label
"""
valid_label = list()
for x in valid_gen:
  valid_label.append(x[1].numpy())
valid_label = np.concatenate(valid_label)

In [ ]:
print(classification_report(y_pred=valid_pred_ids, y_true=valid_label))

              precision    recall  f1-score   support

           0       0.64      0.39      0.48        59
           1       0.97      0.99      0.98      1360

    accuracy                           0.97      1419
   macro avg       0.81      0.69      0.73      1419
weighted avg       0.96      0.97      0.96      1419



In [ ]:
confm = confusion_matrix(y_pred=valid_pred_ids, y_true=valid_label)

index = ['Actual_0', 'Actual_1']
columns = ['Pred_0', 'Pred_1']
pd.DataFrame(confm, index=index, columns=columns)

,Pred_0,Pred_1
Actual_0,23,36
Actual_1,13,1347


## Load model and predict

In [ ]:
new_model = TFBertForSequenceClassification.from_pretrained('save_chinese/')
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

Some layers from the model checkpoint at save_chinese/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at save_chinese/ and are newly initialized: ['dropout_75']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sentence = ["文瑋助教好壯"]

test_dataset = pd.DataFrame(dict(idx=list(range(len(sentence))),
                                 label=[0]*len(sentence),
                                 sentence=sentence))

In [ ]:
test_dataset

,idx,label,sentence
0,0,0,文瑋助教好壯


In [ ]:
test_gen = tf.data.Dataset.from_tensor_slices(dict(test_dataset))

In [ ]:
max_length = 512
task = 'cola'
test_gen = glue_convert_examples_to_features(test_gen, tokenizer, max_length, task)

/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:284: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


In [ ]:
test_gen = test_gen.batch(1)

In [ ]:
next(iter(test_gen))

({'attention_mask': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
  array([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0

In [ ]:
pred = new_model.predict(test_gen)

In [ ]:
pred_ids = np.argmax(pred, axis=-1)

In [ ]:
print(pred_ids[0])

[1]
